In [ ]:
import pandas as pd
import json
import os
import folium
import requests
import time

def visualize_routes():
    print("🗺️  Memulai Visualisasi Peta...")
    
    BASE_DIR = os.getcwd()
    if os.path.basename(BASE_DIR) == 'notebook':
        BASE_DIR = os.path.dirname(BASE_DIR)
    DATA_DIR = os.path.join(BASE_DIR, 'dataset')
    RES_DIR = os.path.join(BASE_DIR, 'results')
    FIG_DIR = os.path.join(BASE_DIR, 'figures')
    os.makedirs(FIG_DIR, exist_ok=True)
    
    CSV_FILE = os.path.join(DATA_DIR, 'final_dataset_2echelon.csv')
    ROUTES_FILE = os.path.join(RES_DIR, 'optimized_routes.json')
    
    if not os.path.exists(ROUTES_FILE):
        print("❌ File hasil rute tidak ditemukan. Jalankan script modelling dulu.")
        return

    # Load Data
    df = pd.read_csv(CSV_FILE)
    nodes_map = df.set_index('Node_ID').to_dict('index')
    
    with open(ROUTES_FILE, 'r') as f:
        routes = json.load(f)
        
    # Setup Peta (Bandung)
    m = folium.Map(location=[-6.9175, 107.6191], zoom_start=13, tiles='CartoDB positron')
    
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']
    
    # 1. Plot Semua Node
    for nid, info in nodes_map.items():
        loc = [info['Lat'], info['Lon']]
        if info['Type'] == 'TPS':
            folium.CircleMarker(
                location=loc, radius=4, color='#333', weight=1, fill=True, fill_color='white', fill_opacity=1,
                popup=f"TPS: {info['Nama Fasilitas']}<br>Sampah: {info.get('Demand_Ton',0)}T<br>Biaya Gerobak: Rp {info.get('Cost_Gerobak',0):,}"
            ).add_to(m)
        elif info['Type'] == 'POOL':
            folium.Marker(
                location=loc, icon=folium.Icon(color='blue', icon='warehouse', prefix='fa'),
                tooltip=f"POOL: {info['Nama Fasilitas']}"
            ).add_to(m)
        elif info['Type'] == 'TPA':
            folium.Marker(
                location=loc, icon=folium.Icon(color='red', icon='trash', prefix='fa'),
                tooltip="TPA SARIMUKTI"
            ).add_to(m)

    # 2. Gambar Rute Jalan Asli (OSRM)
    def get_geometry(waypoints):
        # Format: lon,lat;lon,lat
        coords = ";".join([f"{lon},{lat}" for lat, lon in waypoints])
        url = f"http://router.project-osrm.org/route/v1/driving/{coords}?overview=full&geometries=geojson"
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200:
                geo = r.json()['routes'][0]['geometry']['coordinates']
                return [[y, x] for x, y in geo] # Balik ke lat,lon
        except:
            pass
        return waypoints # Fallback garis lurus

    print(f"🖌️  Menggambar {len(routes)} rute...")
    
    for i, r in enumerate(routes):
        node_ids = r['Route_Nodes']
        waypoints = []
        for nid in node_ids:
            if nid in nodes_map:
                waypoints.append([nodes_map[nid]['Lat'], nodes_map[nid]['Lon']])
        
        if len(waypoints) < 2: continue
        
        # Request OSRM
        path = get_geometry(waypoints)
        color = colors[r['Kecamatan_ID'] % len(colors)]
        
        folium.PolyLine(
            path, color=color, weight=3, opacity=0.8,
            tooltip=f"{r['Vehicle']} (Load: {r['Total_Load']}T)"
        ).add_to(m)
        
        time.sleep(0.1) # Rate limit

    out_map = os.path.join(FIG_DIR, 'peta_optimasi_rute.html')
    m.save(out_map)
    print(f"✅ Peta Visualisasi selesai! Buka file: {out_map}")

if __name__ == "__main__":
    visualize_routes()

🗺️  Memulai Visualisasi Peta...
🖌️  Menggambar 151 rute...
